In [1]:
# # # Recuerda no correr todo a menos que sea necesario
# # # Usa lo ya filtrado
#using Pkg; Pkg.add("CategoricalArrays")

In [1]:
using CSV, DataFrames, StatsBase, GLM, Random, CategoricalArrays

In [2]:
# Equivalente a set.seed
Random.seed!(99)

MersenneTwister(99)

In [3]:
# Base de datos del link https://www.inegi.org.mx/programas/ccpv/2020/default.html#Microdatos
# se tarda como 4 miutos en cargar

personas = CSV.read("Personas00.csv", DataFrame)

,ENT,MUN,LOC50K,ID_VIV,ID_PERSONA,COBERTURA,ESTRATO
,Int64,Int64,Int64,Int64,Int64,Int64,String15
1,1,1,1,10010000001,1001000000100001,2,01-001-0001-00
2,1,1,1,10010000001,1001000000100002,2,01-001-0001-00
3,1,1,1,10010000001,1001000000100003,2,01-001-0001-00
4,1,1,1,10010000002,1001000000200001,2,01-001-0001-00
5,1,1,1,10010000002,1001000000200002,2,01-001-0001-00
6,1,1,1,10010000002,1001000000200003,2,01-001-0001-00
7,1,1,1,10010000002,1001000000200004,2,01-001-0001-00
8,1,1,1,10010000003,1001000000300001,2,01-001-0001-00
9,1,1,1,10010000004,1001000000400001,2,01-001-0001-00


In [4]:
# Las columnas que estoy seleccionando para el ajuste
col_sel = ["ID_PERSONA","ENT","SEXO", "EDAD", "NIVACAD", "ALFABET", 
            "INGTRMEN","HORTRA", "CONACT", "SITTRA", "ENT_PAIS_TRAB",
        "AGUINALDO", "VACACIONES", "SERVICIO_MEDICO", "UTILIDADES", "INCAP_SUELDO", "SAR_AFORE", "CREDITO_VIVIENDA"]

18-element Vector{String}:
 "ID_PERSONA"
 "ENT"
 "SEXO"
 "EDAD"
 "NIVACAD"
 "ALFABET"
 "INGTRMEN"
 "HORTRA"
 "CONACT"
 "SITTRA"
 "ENT_PAIS_TRAB"
 "AGUINALDO"
 "VACACIONES"
 "SERVICIO_MEDICO"
 "UTILIDADES"
 "INCAP_SUELDO"
 "SAR_AFORE"
 "CREDITO_VIVIENDA"

In [5]:
# Selecciono solo las columnas que quiero
personas_filt = personas[:, col_sel]

,ID_PERSONA,ENT,SEXO,EDAD,NIVACAD,ALFABET,INGTRMEN,HORTRA,CONACT
,Int64,Int64,Int64,Int64,Int64?,Int64?,Int64?,Int64?,Int64?
1,1001000000100001,1,3,55,11,1,missing,missing,60
2,1001000000100002,1,1,18,0,3,missing,missing,80
3,1001000000100003,1,1,21,11,1,missing,missing,50
4,1001000000200001,1,1,45,11,1,21500,40,10
5,1001000000200002,1,3,42,11,1,8600,40,10
6,1001000000200003,1,3,20,11,1,missing,missing,50
7,1001000000200004,1,1,10,2,1,missing,missing,missing
8,1001000000300001,1,1,60,11,1,20000,48,10
9,1001000000400001,1,1,62,13,1,30000,21,10


In [6]:
# Es momento de filtrar

# Primero filtramos con las personas que actually trabajan porque son las que nos importan
cond_act = [10, 13, 14, 15, 16, 17, 18, 19, 20]
personas_filt = subset(personas_filt, :CONACT => ByRow(in(cond_act)), skipmissing = true)

# Este filtro se queda porque es menor que 
# Este es para entidad de trabajo
personas_filt = subset(personas_filt, :ENT_PAIS_TRAB => ByRow(<(33)), skipmissing = true)
# y este es para entidad de vivienda
personas_filt = subset(personas_filt, :ENT => ByRow(<(33)), skipmissing = true)

# y este para las horas trabajadas
personas_filt = subset(personas_filt, :HORTRA => ByRow(!=(999)), skipmissing = true)

# y el ingreso ganado
personas_filt = subset(personas_filt, :INGTRMEN => ByRow(!=(999999)), skipmissing = true)


,ID_PERSONA,ENT,SEXO,EDAD,NIVACAD,ALFABET,INGTRMEN,HORTRA,CONACT
,Int64,Int64,Int64,Int64,Int64?,Int64?,Int64?,Int64?,Int64?
1,1001000000200001,1,1,45,11,1,21500,40,10
2,1001000000200002,1,3,42,11,1,8600,40,10
3,1001000000300001,1,1,60,11,1,20000,48,10
4,1001000000400001,1,1,62,13,1,30000,21,10
5,1001000000400002,1,1,32,11,1,4000,48,10
6,1001000000400003,1,3,54,13,1,32000,42,10
7,1001000000500001,1,1,52,11,1,25000,0,20
8,1001000000600002,1,3,41,11,1,25000,40,10
9,1001000000600003,1,1,40,11,1,25000,72,10


In [7]:
function diferente_a(dataframe, columna, condicion)
    dataframe = subset(dataframe, columna => ByRow(!=(condicion)), skipmissing = true)
    return dataframe
end

categorias_9 = ["SEXO", "AGUINALDO", "VACACIONES", "SERVICIO_MEDICO", "UTILIDADES", 
    "INCAP_SUELDO", "SAR_AFORE", "CREDITO_VIVIENDA", "ALFABET", "SITTRA"]

categorias_99 = ["NIVACAD"]

for i = 1:length(categorias_9)
    personas_filt = diferente_a(personas_filt, categorias_9[i], 9)
end

for i = 1:length(categorias_99)
    personas_filt = diferente_a(personas_filt, categorias_99[i], 99)
end

In [8]:
CSV.write("personas_filtradas.csv", personas_filt)

"personas_filtradas.csv"

In [9]:
data = CSV.read("personas_filtradas.csv", DataFrame)

,ID_PERSONA,ENT,SEXO,EDAD,NIVACAD,ALFABET,INGTRMEN,HORTRA,CONACT
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,1001000000400001,1,1,62,13,1,30000,21,10
2,1001000000400002,1,1,32,11,1,4000,48,10
3,1001000000400003,1,3,54,13,1,32000,42,10
4,1001000000600002,1,3,41,11,1,25000,40,10
5,1001000000700001,1,1,28,11,1,40000,60,10
6,1001000000800002,1,3,42,13,1,12000,45,10
7,1001000000800003,1,1,42,13,1,16000,45,10
8,1001000000900003,1,1,45,11,1,20000,42,10
9,1001000001200005,1,3,43,13,1,20000,40,10


In [10]:
# Vector con todas las categorias
vector_categorias = ["SEXO", "AGUINALDO", "VACACIONES", "SERVICIO_MEDICO", "UTILIDADES", 
    "INCAP_SUELDO", "SAR_AFORE", "CREDITO_VIVIENDA", "ALFABET", "NIVACAD", "ENT_PAIS_TRAB", 
    "ENT", "SITTRA"]

transform!(data, names(data, vector_categorias) .=> categorical, renamecols=false)

,ID_PERSONA,ENT,SEXO,EDAD,NIVACAD,ALFABET,INGTRMEN,HORTRA,CONACT
,Int64,Cat…,Cat…,Int64,Cat…,Cat…,Int64,Int64,Int64
1,1001000000400001,1,1,62,13,1,30000,21,10
2,1001000000400002,1,1,32,11,1,4000,48,10
3,1001000000400003,1,3,54,13,1,32000,42,10
4,1001000000600002,1,3,41,11,1,25000,40,10
5,1001000000700001,1,1,28,11,1,40000,60,10
6,1001000000800002,1,3,42,13,1,12000,45,10
7,1001000000800003,1,1,42,13,1,16000,45,10
8,1001000000900003,1,1,45,11,1,20000,42,10
9,1001000001200005,1,3,43,13,1,20000,40,10


In [11]:
### FIT BASE ###
function fit5(cantidad_sample, nombre_facil)
    nombre_fit = "fit5"
    
    sample_rows = sample(1:nrow(data), cantidad_sample, replace=false)

    df_sample = data[sample_rows, :]
    
    nombre_completo = nombre_facil*"_"*nombre_fit*".csv"
    # Guardamos el documentos para usarlo en R
    CSV.write(nombre_completo, df_sample)

    # Hacemos el fit
    sample_fit = lm(@formula(INGTRMEN ~ SEXO + EDAD + NIVACAD + HORTRA + ENT_PAIS_TRAB), df_sample)

    aux = "res_"
    nombre_completo = aux*nombre_completo
    CSV.write(nombre_completo, coeftable(sample_fit))
end 

fit5 (generic function with 1 method)

In [12]:
# Fit 5
fit5(500, "500")
fit5(5000, "5mil")
fit5(50000, "50mil")
fit5(500000, "500mil")
fit5(2500000, "2500mil")

"res_2500mil_fit5.csv"

In [13]:
### FIT 6 ###
function fit6(cantidad_sample, nombre_facil)
    nombre_fit = "fit6"
    
    sample_rows = sample(1:nrow(data), cantidad_sample, replace=false)

    df_sample = data[sample_rows, :]
    
    nombre_completo = nombre_facil*"_"*nombre_fit*".csv"
    # Guardamos el documentos para usarlo en R
    CSV.write(nombre_completo, df_sample)

    # Hacemos el fit
    sample_fit = lm(@formula(INGTRMEN ~ SEXO + EDAD + NIVACAD + HORTRA + ENT_PAIS_TRAB + SITTRA), df_sample)

    aux = "res_"
    nombre_completo = aux*nombre_completo
    CSV.write(nombre_completo, coeftable(sample_fit))
end 

fit6 (generic function with 1 method)

In [14]:
# Fit 6: Fit 5 + SITTRA
fit6(500, "500")
fit6(5000, "5mil")
fit6(50000, "50mil")
fit6(500000, "500mil")
fit6(2500000, "2500mil")

"res_2500mil_fit6.csv"

In [15]:
### FIT 7 ###
function fit7(cantidad_sample, nombre_facil)
    nombre_fit = "fit7"
    
    sample_rows = sample(1:nrow(data), cantidad_sample, replace=false)

    df_sample = data[sample_rows, :]
    
    nombre_completo = nombre_facil*"_"*nombre_fit*".csv"
    # Guardamos el documentos para usarlo en R
    CSV.write(nombre_completo, df_sample)

    # Hacemos el fit
    sample_fit = lm(@formula(INGTRMEN ~ SEXO + EDAD + NIVACAD + HORTRA + ENT_PAIS_TRAB
            + SITTRA + ALFABET), df_sample)

    aux = "res_"
    nombre_completo = aux*nombre_completo
    CSV.write(nombre_completo, coeftable(sample_fit))
end 

fit7 (generic function with 1 method)

In [16]:
# Fit 7: Fit 5 + SITTRA + ALFABET
fit7(500, "500")
fit7(5000, "5mil")
fit7(50000, "50mil")
fit7(500000, "500mil")
fit7(2500000, "2500mil")

"res_2500mil_fit7.csv"

In [17]:
### FIT 8###
function fit8(cantidad_sample, nombre_facil)
    nombre_fit = "fit8"
    
    sample_rows = sample(1:nrow(data), cantidad_sample, replace=false)

    df_sample = data[sample_rows, :]
    
    nombre_completo = nombre_facil*"_"*nombre_fit*".csv"
    # Guardamos el documentos para usarlo en R
    CSV.write(nombre_completo, df_sample)

    # Hacemos el fit
    sample_fit = lm(@formula(INGTRMEN ~ SEXO + EDAD + NIVACAD + HORTRA + ENT_PAIS_TRAB
            + SITTRA + ALFABET + AGUINALDO), df_sample)

    aux = "res_"
    nombre_completo = aux*nombre_completo
    CSV.write(nombre_completo, coeftable(sample_fit))
end 

fit8 (generic function with 1 method)

In [18]:
# Fit 8: Fit 5 + SITTRA + ALFABET + AGUINALDO
fit8(500, "500")
fit8(5000, "5mil")
fit8(50000, "50mil")
fit8(500000, "500mil")
fit8(2500000, "2500mil")

"res_2500mil_fit8.csv"

In [19]:
### FIT 9###
function fit9(cantidad_sample, nombre_facil)
    nombre_fit = "fit9"
    
    sample_rows = sample(1:nrow(data), cantidad_sample, replace=false)

    df_sample = data[sample_rows, :]
    
    nombre_completo = nombre_facil*"_"*nombre_fit*".csv"
    # Guardamos el documentos para usarlo en R
    CSV.write(nombre_completo, df_sample)

    # Hacemos el fit
    sample_fit = lm(@formula(INGTRMEN ~ SEXO + EDAD + NIVACAD + HORTRA + ENT_PAIS_TRAB
            + SITTRA + ALFABET + AGUINALDO + VACACIONES), df_sample)

    aux = "res_"
    nombre_completo = aux*nombre_completo
    CSV.write(nombre_completo, coeftable(sample_fit))
end 

fit9 (generic function with 1 method)

In [20]:
# Fit 9: Fit 5 + SITTRA + ALFABET + AGUINALDO +VACACIONES
fit9(500, "500")
fit9(5000, "5mil")
fit9(50000, "50mil")
fit9(500000, "500mil")
fit9(2500000, "2500mil")

"res_2500mil_fit9.csv"

In [21]:
### FIT 10###
function fit10(cantidad_sample, nombre_facil)
    nombre_fit = "fit10"
    
    sample_rows = sample(1:nrow(data), cantidad_sample, replace=false)

    df_sample = data[sample_rows, :]
    
    nombre_completo = nombre_facil*"_"*nombre_fit*".csv"
    # Guardamos el documentos para usarlo en R
    CSV.write(nombre_completo, df_sample)

    # Hacemos el fit
    sample_fit = lm(@formula(INGTRMEN ~ SEXO + EDAD + NIVACAD + HORTRA + ENT_PAIS_TRAB
            + SITTRA + ALFABET + AGUINALDO + VACACIONES + SERVICIO_MEDICO), df_sample)

    aux = "res_"
    nombre_completo = aux*nombre_completo
    CSV.write(nombre_completo, coeftable(sample_fit))
end 

fit10 (generic function with 1 method)

In [22]:
# Fit 10: Fit 5 + SITTRA + ALFABET + AGUINALDO + VACACIONES + SERVICIO_MEDICO
fit10(500, "500")
fit10(5000, "5mil")
fit10(50000, "50mil")
fit10(500000, "500mil")
fit10(2500000, "2500mil")

"res_2500mil_fit10.csv"